# Pythonクライアントを用いた動作確認

In [5]:
import delta_sharing

In [6]:
profile_file = "deltasharing.json"

クライアント生成

In [7]:
client = delta_sharing.SharingClient(profile_file)

In [8]:
client.list_all_tables()

[Table(name='table1', share='share1', schema='schema1'),
 Table(name='table2', share='share1', schema='schema1')]

テーブルにアクセスする

In [11]:
table_url = profile_file + "#share1.schema1.table1"

In [12]:
delta_sharing.load_as_pandas(table_url)

,id
0,7
1,1
2,2
3,8
4,0
5,9
6,6
7,4
8,3
9,5


## トラブルシュートメモ

### S3互換ストレージの対応状況

とりあえず、サーバ起動時にAWSのクレデンシャルを環境変数で与えたが、S3エンドポイントの設定をどうすればよいのか。
サーバのリファレンス実装がHadoopのコンフィグレーションを渡すようになっておらず…。という状況だった。

ひとまず、暫定的にサーバの実装を以下のようにしたら動いた。

io/delta/standalone/internal/DeltaSharedTableLoader.scala:70
```scala
  private val conf = withClassLoader {
    val conf = new Configuration()
    conf.set("fs.s3a.endpoint", "http://127.0.0.1:9000")
    conf.set("fs.s3a.aws.credentials.provider",
      "com.amazonaws.auth.profile.ProfileCredentialsProvider")
    conf
  }
```

あるべき姿という意味では、コンフィグを読み込むようにすべきであるか。

### 追記（2021/7/12）

最近 https://github.com/delta-io/delta-sharing/pull/45 によって conf/ 以下がクラスパスに渡されるようになったようだ。
なので、Delta Sharingのサーバを起動する際、conf/以下にcore-site.xmlを置き、 https://hadoop.apache.org/docs/current/hadoop-aws/tools/hadoop-aws/index.html に記載されているようなパラメータを設定すればよくなった。